## Ambiente Destinados às análises finais

In [1]:
%defaultDatasource jdbc:h2:mem:db

Tabela das Receitas

In [43]:
DROP TABLE IF EXISTS Recipes

In [44]:
CREATE TABLE Recipes AS 
SELECT "Recipe ID" AS recipe_id, Cuisine, Title
FROM CSVREAD('../data/raw/01_Recipe_Details.csv');

In [61]:
DROP INDEX IF EXISTS index_R

In [62]:
CREATE INDEX index_R
ON Recipes (recipe_id, Cuisine, Title);

Tabela dos Ingredientes

In [5]:
DROP TABLE IF EXISTS Ingredients_to_Foodgroup

In [6]:
CREATE TABLE Ingredients_to_Foodgroup AS 
SELECT ID, food_group
FROM CSVREAD('../data/interim/Ingredients_to_Foodb_Groups.csv');

In [7]:
CREATE INDEX index_IF
ON Ingredients_to_Foodgroup (ID);

Tabela de ligação entre receitas e ingredientes

In [8]:
DROP TABLE IF EXISTS Recipes_Ingredients

In [9]:
CREATE TABLE Recipes_Ingredients AS
SELECT "Recipe ID" AS recipe_id, "Entity ID" as ingredient_id
FROM CSVREAD('../data/raw/04_Recipe-Ingredients_Aliases.csv');

In [10]:
CREATE INDEX index_RI
ON Recipes_Ingredients (recipe_id);

Tabela de ligação entre Foodb e ingredientes

Tabela de ligação entre país e região

In [40]:
DROP TABLE IF EXISTS COUNTRIES

In [41]:
CREATE TABLE Countries AS
SELECT Country, "World Region" AS region
FROM CSVREAD('../data/interim/CountriesTable.csv');

In [42]:
CREATE INDEX index_countries
ON Countries(country, region)

Criando tabela do food prices

In [13]:
DROP TABLE IF EXISTS prices

In [14]:
CREATE TABLE Prices AS
SELECT "Country Name" AS country, "Series Name" AS category, "2017 [YR2017]" AS price
FROM CSVREAD('../data/raw/Food_Prices_Data.csv');

In [29]:
CREATE INDEX index_P
ON Prices(country, category)

Atualizando os valores dos preços para float, e colocando como 0 os que não tem dados

In [15]:
UPDATE Prices
SET price = CASE 
                    WHEN price = '..' THEN 0
                    WHEN price NOT LIKE '%[^0-9.]%' THEN CAST(price AS FLOAT)
                    ELSE price
                END;


Criando uma tabela de preços de categoria por região

In [16]:
DROP TABLE IF EXISTS Prices_per_Region

In [17]:
CREATE TABLE Prices_per_Region AS 
SELECT DISTINCT C.region, AVG(COALESCE(CAST(price AS FLOAT), 0)) AS average_price, P.category
FROM Countries C, Prices P
WHERE C.Country = P.country 
GROUP BY C.region, P.category
ORDER BY region

In [18]:
CALL CSVWRITE('../data/interim/Category_Prices_per_Region.csv', 'SELECT * FROM Prices_per_Region');

120

Criando tabela de Receitas por Região

In [19]:
DROP TABLE Foodb_to_Prices IF EXISTS

Carregando a tabela de Foodb_to_Prices

In [20]:
CREATE TABLE Foodb_to_Prices AS
SELECT *
FROM CSVREAD('../data/interim/Foodb_to_Food_Prices.csv');

Criando uma tabela que, para cada ingrediente, para cada receita, temos a categoria do ingrediente (para podermos analisar o seu preço)

In [21]:
CREATE TABLE Ingredients_Series_per_Recipes AS
SELECT RI.recipe_id, FP.series_name
FROM Recipes_Ingredients RI, Ingredients_to_Foodgroup IF, Foodb_to_Prices FP
WHERE RI.ingredient_id = IF.id AND IF.food_group=FP.food_group 

In [22]:
CREATE INDEX index_ISR
ON Ingredients_Series_per_Recipes (recipe_id,series_name);

In [27]:
DROP TABLE IF EXISTS Recipe_Price_per_Region

Criando tabela com todos os preços de todas as receitas por região

In [28]:
CREATE TABLE Recipe_Price_per_Region AS
SELECT ISR.recipe_id, RA.Title, RA.cuisine, SUM(PR.average_price) as recipe_price
FROM Ingredients_Series_per_Recipes ISR, Recipes RA, Prices_per_Region PR
WHERE RA.recipe_id=ISR.recipe_id AND (PR.region=RA.cuisine) AND (ISR.series_name=PR.category)
GROUP BY ISR.recipe_ID
ORDER BY RA.cuisine, ISR.recipe_id

In [25]:
CALL CSVWRITE('../data/processed/Recipe_Price_per_Region.csv', 'SELECT * FROM Recipe_Price_per_Region');

44023

Criando tabela para saber o preço de receitas por país

In [70]:
DROP TABLE IF EXISTS Recipes_Prices_per_Country

In [71]:
CREATE TABLE Recipes_Prices_per_Country AS
SELECT IR.recipe_id, R.Title, P.country, SUM(COALESCE(CAST(price AS FLOAT), 0)) as price
FROM Prices P, Ingredients_Series_per_Recipes IR, Countries C, Recipes R
WHERE P.category = IR.series_name AND R.Cuisine = C.region AND C.country = P.country AND R.recipe_id = IR.recipe_id
GROUP BY IR.recipe_id, P.country
ORDER BY R.Title, P.country

In [72]:
CALL CSVWRITE('../data/processed/Recipe_Price_per_Country.csv', 'SELECT * FROM Recipes_Prices_per_Country');

107529